# Тема: Автоматизация отчетности
# Проект: Создание дашборда с метриками взаимодействия пользователей с карточками статей

# Техническое задание:
* Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;

* Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;

* Кто будет основным пользователем дашборда: менеджеры по анализу контента;

* Состав данных для дашборда:
    * История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
    * Разбивка событий по темам источников;
    * Таблица соответствия тем источников темам карточек;
* По каким параметрам данные должны группироваться:
    * Дата и время;
    * Тема карточки;
    * Тема источника;
    * Возрастная группа;

* Характер данных:
    * История событий по темам карточек — абсолютные величины с разбивкой по минутам;
    * Разбивка событий по темам источников — относительные величины (% событий);
    * Соответствия тем источников темам карточек - абсолютные величины;

* Важность: все графики имеют равную важность;

* Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);

* База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД zen;

* Частота обновления данных: один раз в сутки, в полночь по UTC;

* Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (макет дашборда):

![dashboard_example](dashboard_example.png)

# Материалы:

[Дашборд](https://public.tableau.com/views/Dashboard_Yandex_Zen_16555074175810/sheet5?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)

[Презентация](https://disk.yandex.ru/i/AE2om-fviyc9Gw)

## Загрузка данных и подготовка к анализу

In [1]:
# установливаем пакета для работы с PostgreSQL
!pip install psycopg2-binary

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [3]:
# cоздаём коннекцию к базе
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

In [4]:
# формируем sql-запрос
query = ''' SELECT * 
            FROM dash_visits
        '''

In [5]:
# выполняем sql-запрос
dash_visits = pd.io.sql.read_sql(query, con = engine)

In [6]:
# выгружаем данные из таблицы dash_visits в файл dash_visits.csv
dash_visits.to_csv('dash_visits.csv', index=False)

In [7]:
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [9]:
# уточним период исследования
dash_visits['dt'].min(), dash_visits['dt'].max()

(Timestamp('2019-09-24 18:28:00'), Timestamp('2019-09-24 19:00:00'))

In [10]:
# выясним общее количество визитов за период
dash_visits['visits'].sum()

310207

In [16]:
# итоговое количество уникальных тем карточек
dash_visits['item_topic'].nunique()

25

In [17]:
# итоговое количество уникальных ресурсов
dash_visits['source_topic'].nunique()

26